# MSc Research Project - Coding Exercise - Regression

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.stats import pearsonr
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.neural_network import MLPRegressor
from pathlib import Path

## Load dataset

In [ ]:
filepath = Path().absolute() / 'four_outputs_liqcf_pacific.csv'
df = pd.read_csv(filepath)

## Extract features and target

In [ ]:
features = ['tot_aod', 'RH700', 'RH850', 'w500', 'whoi_sst']
target_name = 'cod'
X, y = df[features], df[target_name]

## Split data

In [ ]:
def data_split_no_sk(X, y, train_size, valid_size, test_size):
    assert(train_size+valid_size+test_size == 1)
    assert(len(X) == len(y))
    N = len(X)
    train_idx, val_idx, test_idx  = np.split(np.random.permutation(N), [int(N*train_size), int(N*(train_size+valid_size))])
    return X.iloc[train_idx], y.iloc[train_idx], X.iloc[val_idx], y.iloc[val_idx], X.iloc[test_idx], y.iloc[test_idx]

In [ ]:
def data_split_sk(X, y, train_size, valid_size, test_size):
    assert(train_size+valid_size+test_size == 1)
    assert(len(X) == len(y))
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=valid_size/(train_size+valid_size))
    return X_train, y_train, X_val, y_val, X_test, y_test

## Models

### LinearRegression

In [ ]:
def train_test_linreg(X_train, y_train, X_test, y_test):
    """
    Input: training and testing sets
    Output: coefficient of determination of the prediction, R^2,
            we want it to be as closed to 1.0 as possible
    """
    pipeline = make_pipeline(StandardScaler(), LinearRegression())
    pipeline.fit(X_train, y_train)
    return pipeline.score(X_test, y_test)

def complete_linreg(X, y):
    scores = []
    for _ in range(10):
        X_train, y_train, _, _, X_test, y_test = data_split_no_sk(X, y, 0.9, 0, 0.1)
        score = train_test_linreg(X_train, y_train, X_test, y_test)
        scores.append(score)
    return scores

scores_linreg = complete_linreg(X, y)
scores_linreg_avg = np.mean(scores_linreg)
print(scores_linreg, scores_linreg_avg)
    

### Polynomial Basis Expansion

In [ ]:
def train_test_polyreg(X_train, y_train, X_test, y_test, deg):
    """
    Input: training set, testing set, and degree for polynomial expansion
    Output: coefficient of determination of the prediction, R^2,
            we want it to be as closed to 1.0 as possible
    """
    pipeline = make_pipeline(PolynomialFeatures(degree=deg), StandardScaler(), LinearRegression())
    # poly = PolynomialFeatures(degree=deg)
    # X_train_ = poly.fit_transform(X_train)
    # X_test_ = poly.fit_transform(X_test)
    # model = LinearRegression()
    pipeline.fit(X_train, y_train)
    return pipeline.score(X_test, y_test)

def fit_parameters_polyreg(X, y):
    scores = []
    X_train, y_train, _, _, X_test, y_test = data_split_no_sk(X, y, 0.9, 0, 0.1)
    for deg in range(1, 6):
        score = train_test_polyreg(X_train, y_train, X_test, y_test, deg)
        scores.append(score)
    return scores

def complete_polyreg(X, y, deg):
    scores = []
    for _ in range(10):
        X_train, y_train, _, _, X_test, y_test = data_split_no_sk(X, y, 0.9, 0, 0.1)
        score = train_test_polyreg(X_train, y_train, X_test, y_test, deg)
        scores.append(score)
    return scores

# scores = fit_parameters_polyreg(X, y)
# print(scores)

full_scores_polyreg = complete_polyreg(X, y, 5)
scores_polyreg_avg = np.mean(full_scores_polyreg)
print(full_scores_polyreg, scores_polyreg_avg)


### Neural Network

In [ ]:
def train_val_test_mlp1(X_train, y_train, X_test, y_test):
    """
    Input: training (including validation) and testing sets
    Output: coefficient of determination of the prediction, R^2,
            we want it to be as closed to 1.0 as possible
    """
    pipeline = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100, ), activation="relu", solver="adam", early_stopping=True, validation_fraction=0.1, verbose=True, n_iter_no_change=2))
    pipeline.fit(X_train.values, y_train)
    return pipeline.score(X_test.values, y_test)

def complete_mlp1(X, y):
    scores = []
    for _ in range(10):
        X_train, y_train, _, _, X_test, y_test = data_split_no_sk(X, y, 0.9, 0, 0.1)
        score = train_val_test_mlp1(X_train, y_train, X_test, y_test)
        scores.append(score)
    return scores

scores_mlp1 = complete_mlp1(X, y)
scores_mlp1_avg = np.mean(scores_mlp1)
print(scores_mlp1, scores_mlp1_avg)


In [ ]:
def train_val_test_mlp2(X_train, y_train, X_test, y_test):
    """
    Input: training (including validation) and testing sets
    Output: coefficient of determination of the prediction, R^2,
            we want it to be as closed to 1.0 as possible
    """
    pipeline = make_pipeline(StandardScaler(), MLPRegressor(hidden_layer_sizes=(100, 50, ), activation="relu", solver="adam", early_stopping=True, validation_fraction=0.1, verbose=True, n_iter_no_change=2))
    pipeline.fit(X_train.values, y_train)
    return pipeline.score(X_test.values, y_test)

def complete_mlp2(X, y):
    scores = []
    for _ in range(10):
        X_train, y_train, _, _, X_test, y_test = data_split_no_sk(X, y, 0.9, 0, 0.1)
        score = train_val_test_mlp2(X_train, y_train, X_test, y_test)
        scores.append(score)
    return scores

scores_mlp2 = complete_mlp2(X, y)
scores_mlp2_avg = np.mean(scores_mlp2)
print(scores_mlp2, scores_mlp2_avg)